In [4]:
from datasets import concatenate_datasets
from datasets import load_dataset
import torch
import json
from dataclasses import dataclass, field
import re
from typing import Optional
import os
import evaluate
import torch
from datasets import concatenate_datasets
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from datasets import load_from_disk
import pickle
import pandas as pd
import random
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed
from trl.core import LengthSampler
tqdm.pandas()

In [16]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
toxicity = evaluate.load("ybelkada/toxicity", "DaNLP/da-electra-hatespeech-detection", module_type="measurement")
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, cache_dir='/home/chenboc1/localscratch2/chenboc1/trl/.cache').to(device)


In [5]:
FULL_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/result.json'
JOE_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/prompt_selected/Joe_Biden_response.json'
IPHONE_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/prompt_selected/Iphone_response.json'

In [6]:

with open(FULL_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    FULL_RESULT = json.load(json_file)

with open(JOE_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    JOE_RESULT = json.load(json_file)

with open(IPHONE_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    IPHONE_RESULT = json.load(json_file)


In [7]:
FULL_RESULT.keys()

dict_keys(['ctx', 'real'])

In [8]:
JOE_RESULT.keys()

dict_keys(['ctx_Joe_Biden', 'realtox_Joe_Biden'])

In [9]:
IPHONE_RESULT.keys()

dict_keys(['ctx_Joe_Biden', 'realtox_Joe_Biden'])

In [10]:
FULL_RESULT['ctx'].keys()

dict_keys(['prompt', 'gpt_generation', 'DaNLP_score', 'roberta_score'])

In [11]:
FULL_RESULT['ctx'].update({'DaNLP_score_Gen':[], 'roberta_score_Gen': []})
FULL_RESULT['real'].update({'DaNLP_score_Gen':[], 'roberta_score_Gen': []})

In [12]:
FULL_RESULT['real']['DaNLP_score_Gen']

[]

In [13]:
FULL_RESULT['real'].keys()

dict_keys(['prompt', 'gpt_generation', 'DaNLP_score', 'roberta_score', 'DaNLP_score_Gen', 'roberta_score_Gen'])

In [28]:
response_column = FULL_RESULT['ctx']['gpt_generation']
input_texts = []
BATCH_SIZE = 256
for i, example in enumerate(response_column):
    input_text = example
    input_texts.append(input_text[:2000])

    if (i + 1) % BATCH_SIZE == 0:
        toxicity_score = toxicity.compute(predictions=input_texts)['toxicity']
        FULL_RESULT['ctx']['DaNLP_score_Gen'].extend(toxicity_score)
        toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(
            device
        )
        logits = toxicity_model(**toxicity_inputs).logits.float()
        toxicity_labels = (logits[:, 0]).tolist()        

        FULL_RESULT['ctx']['roberta_score_Gen'].extend(toxicity_labels)        
        input_texts = []
    if (i + 1) % (BATCH_SIZE*5) == 0:
        print("here is epoch i")



here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i
here is epoch i


In [ ]:
response_column = FULL_RESULT['real']['gpt_generation']
input_texts = []
for i, example in enumerate(response_column):
    input_text = example
    input_texts.append(input_text[:2000])

    if (i + 1) % BATCH_SIZE == 0:
        toxicity_score = toxicity.compute(predictions=input_texts)['toxicity']
        FULL_RESULT['real']['DaNLP_score_Gen'].extend(toxicity_score)
        toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(
            device
        )
        logits = toxicity_model(**toxicity_inputs).logits.float()
        toxicity_labels = (logits[:, 0]).tolist()        

        FULL_RESULT['real']['roberta_score_Gen'].extend(toxicity_labels)        
        input_texts = []
    if (i + 1) % (BATCH_SIZE*5) == 0:
        print("here is epoch i")



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   input_texts.append(input_text[:2000])                                                   │
│    6 │                                                                                           │
│    7 │   if (i + 1) % BATCH_SIZE == 0:                                                           │
│ ❱  8 │   │   toxicity_score = toxicity.compute(predictions=input_texts)['toxicity']              │
│    9 │   │   data_dict['real']['DaNLP_score_Gen'].extend(toxicity_score)                         │
│   10 │   │   toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True,    │
│   11 │   │   │   device                                                                          │
│                                                                                                  │
│ /localscratch/chenboc1/anaconda3/envs/trl/lib/python3.11/site-packages/evaluate/module.py:444 in │
│ compute                                                                                          │
│                                                                                                  │
│   441 │   │   │                                                                                  │
│   442 │   │   │   inputs = {input_name: self.data[input_name] for input_name in self._feature_   │
│   443 │   │   │   with temp_seed(self.seed):                                                     │
│ ❱ 444 │   │   │   │   output = self._compute(**inputs, **compute_kwargs)                         │
│   445 │   │   │                                                                                  │
│   446 │   │   │   if self.buf_writer is not None:                                                │
│   447 │   │   │   │   self.buf_writer = None                                                     │
│                                                                                                  │
│ /home/chenboc1/.cache/huggingface/modules/evaluate_modules/metrics/ybelkada--toxicity/337c291851 │
│ f594c414c4a2027b7901402090d39d0168ce0a518a4019a76b7ca7/toxicity.py:135 in _compute               │
│                                                                                                  │
│   132 │   │   self.toxic_classifier = pipeline("text-classification", model=model_name, top_k=   │
│   133 │                                                                                          │
│   134 │   def _compute(self, predictions, aggregation="all", toxic_label="offensive", threshol   │
│ ❱ 135 │   │   scores = toxicity(predictions, self.toxic_classifier, toxic_label)                 │
│   136 │   │   if aggregation == "ratio":                                                         │
│   137 │   │   │   return {"toxicity_ratio": sum(i >= threshold for i in scores) / len(scores)}   │
│   138 │   │   elif aggregation == "maximum":                                                     │
│                                                                                                  │
│ /home/chenboc1/.cache/huggingface/modules/evaluate_modules/metrics/ybelkada--toxicity/337c291851 │
│ f594c414c4a2027b7901402090d39d0168ce0a518a4019a76b7ca7/toxicity.py:103 in toxicity               │
│                                                                                                  │
│   100 │   │   │   "The `toxic_label` that you specified is not part of the model labels. Run `   │
│   101 │   │   )                                                                                  │
│   102 │                                                                                          │
│ ❱ 103 │   for pred_toxic in toxic_classifier(preds):       

In [ ]:
with open('full_data.json', 'w') as json_file:
    # Load the JSON data into a Python dictionary
    json.dump(FULL_RESULT, json_file)